In [4]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
sys.path.append('script')
from script import dbconn
pgconn = dbconn.db_connection_psycopg()

In [13]:
# Fectching data from the postgreSql database and put the value on raw_df
raw_df = dbconn.db_read_table_psycopg(pgconn,'xdr_data')

c:\Users\o876\Desktop\User-Analytics-in-telecommunication-industry\script\dbconn.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query(sql,pgconn)


In [12]:
# User Overview analysis

In [ ]:
# Identifying the top 10 handsets used by the customers.
# Count handset usage
handset_counts = raw_df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')

# Sort the results
sorted_handsets = handset_counts.sort_values(by='Count', ascending=False)

# Select the top 10 handsets
top_10_handsets = sorted_handsets.head(10)

# Print the top 10 handsets
print(top_10_handsets)

In [ ]:
# Identifying Top 3 handset manufacturers
manufacturer_counts = raw_df['Handset Manufacturer'].value_counts().reset_index()
manufacturer_counts.columns = ['Handset Manufacturer', 'Count']

# Sort the results
sorted_manufacturers = manufacturer_counts.sort_values(by='Count', ascending=False)

# Select the top 3 handset manufacturers
top_3_manufacturers = sorted_manufacturers.head(3)

# Print the top 3 handset manufacturers
print(top_3_manufacturers)

In [ ]:
# Identify the top 5 handsets per top 3 handset manufacturer
# Count handset manufacturers and types
manufacturer_type_counts = raw_df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='Count')

# Sort the results within each manufacturer
sorted_manufacturer_types = manufacturer_type_counts.groupby('Handset Manufacturer').apply(lambda x: x.nlargest(5, 'Count')).reset_index(drop=True)

# Print the top 5 handsets per top 3 handset manufacturers
top_3_manufacturers = sorted_manufacturer_types['Handset Manufacturer'].unique()[:3]

for manufacturer in top_3_manufacturers:
    print(f"Top 5 handsets for {manufacturer}:")
    manufacturer_handsets = sorted_manufacturer_types[sorted_manufacturer_types['Handset Manufacturer'] == manufacturer]
    print(manufacturer_handsets)
    print()

In [23]:
# Identifying users’ behaviour on those Applications / Social Media, Google, Email, Youtube, Netflix, Gaming, Other.   

In [ ]:
# Aggregate the number of xDR sessions per user
sessions_per_user = raw_df.groupby('MSISDN/Number')['Bearer Id'].count()
print(sessions_per_user)

In [ ]:
# Aggregate the session duration per user
session_duration_per_user = raw_df.groupby('MSISDN/Number')['Dur. (ms)'].sum()

# Display the result
print(session_duration_per_user)

In [ ]:
# Aggregate the total download (Dl) and upload data per user
total_data_per_user = raw_df.groupby('MSISDN/Number')[['Total DL (Bytes)', 'Total UL (Bytes)']].sum()

# Display the result
print(total_data_per_user)

In [ ]:
# Create a list of application columns
applications = ['Social Media', 'Google', 'Email', 'Youtube', 'Netflix', 'Gaming', 'Other']

# Aggregate the total data volume per user and application
total_data_per_user_app = raw_df.groupby('MSISDN/Number')[[col + ' DL (Bytes)' for col in applications] + [col + ' UL (Bytes)' for col in applications]].sum()

# Display the result
print(total_data_per_user_app)

In [47]:
# Explanatory Data Analysis | EDA
# Treat all missing values and outliers in the dataset by replacing by the mean of the corresponding column.

In [ ]:
# First Identifying and Treating missing value 
missing_values = raw_df.isna().sum()
print (missing_values)

for column in missing_values.index:
    if missing_values[column] > 0:
        mean_value = str(raw_df[column].mean())
        raw_df[column] = raw_df[column].fillna(mean_value)

In [ ]:
# Analyze the basic metrics (mean, median, etc) from the dataset
raw_df.describe()

In [ ]:
# Conduct a Non-Graphical Univariate Analysis by computing dispersion parameters for each quantitative variable.

quantitative_variables = []

# Iterate over each column in the dataset to find the quantitative variable
for column in raw_df.columns:   
    if raw_df[column].dtype in [int, float]:
        quantitative_variables.append(column)

# Solution: By calculating the range or Difference b/n max and min value in each variable
for column_name in quantitative_variables:
    column_data = raw_df[column_name]
    data_range = column_data.max() - column_data.min()
    print("Range of", column_name, ":", data_range)

In [ ]:
# Conduct a Graphical Univariate Analysis by identifying the most suitable plotting options for each variable and interpret your findings.
# column_name = 'Avg RTT DL (ms)'
clean_Data = raw_df.dropna()
column_names = clean_Data.columns

for column_name in column_names:
    column_data = clean_Data[column_name]    
    plt.hist(column_data, bins=10)
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + column_name)
    plt.show()

In [69]:
# Variable transformations
# Segment the users into top five decile classes based on the total duration for all sessions and compute the total data (DL+UL) per decile class. 

# Calculate the total duration for all sessions for each user
user_total_duration = raw_df.groupby('MSISDN/Number')['Dur. (ms)'].sum()

# Segment users into decile classes
user_deciles = pd.qcut(user_total_duration, q=10, labels=False, duplicates='drop')

# Compute the total data (DL+UL) per decile class
data_per_decile = raw_df.groupby(user_deciles)[['Total DL (Bytes)', 'Total UL (Bytes)']].sum().reset_index()

In [ ]:
# Correlation Analysis – compute a correlation matrix for the following variables and interpret your findings: Social Media data, Google data, Email data, Youtube data, Netflix data, Gaming data, Other data 
# Select the columns for correlation analysis
columns = [
    'Social Media DL (Bytes)', 'Social Media UL (Bytes)',
    'Google DL (Bytes)', 'Google UL (Bytes)',
    'Email DL (Bytes)', 'Email UL (Bytes)',
    'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
    'Netflix DL (Bytes)', 'Netflix UL (Bytes)',
    'Gaming DL (Bytes)', 'Gaming UL (Bytes)',
    'Other DL (Bytes)', 'Other UL (Bytes)'
]

# Create a subset dataframe with the selected columns
subset_df = raw_df[columns]

# Compute the correlation matrix
correlation_matrix = subset_df.corr()

# Display the correlation matrix
print(correlation_matrix)